In [2]:
'''
Considerations
-train neural net within ages of 2-6 Gyrs: we know M67 has an age of about 4Gyrs, so there is no point to training it outside this range
-We want to have a working neural net that works to a precision of 0.005 dex (0.5% precision), though we should make a determination of how accurate we want it to be. 
-Most logical loss function is MAE (mean absolute error), although the Median Absolute Error is smaller. We probably want to train using the Mean Absolute Error and then evaluate using the Median Absolute Error.
-To determine an adequate guess than the number of epochs to train for we can take a small portion of the grid (e.g. say only where the metallicity is solar and take a few masses) train the neural net till it reaches the desired precision, suggested architecture (3,8,8,2) = (inputs,nodes, hidden layers, outputs).
-Sensible number of epochs = 50,000
-We want to try an architecture which is the smallest we can get away with such that its faster (start with small architecture and work up).
'''
#import seaborn as sns
#import pandas as pd
#import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import yaml
#data = np.genfromtxt('grid_0_0.csv',delimiter=",",skip_header=1)
#data = np.genfromtxt('grid_mid_0_0.csv',delimiter=",",skip_header=1)


'''
small grid:
[0]=row
[1]=model_number
[2]=star_mass
[3]=star_age
[4]=star_feh
[5]=star_MLT
[6]=star_fov
[7]=effective_T
[8]=luminosity
[9]=radius
[10]=log_g
[11]=surface_h1
[12]=surface_he4
[13]=center_h1
[14]=center_he4
[15]=log_center_T
[16]=log_center_Rho
[17]=he_core_mass
[18]=delta_nu
[19]=delta_Pg
[20]=nu_max
[21]=nu_0_10
[22]=nu_0_11
[23]=nu_0_12
[24]=nu_0_13
[25]=nu_0_14
[26]=nu_0_15

mid grid:
[0]=row
[1]=star_mass
[2]=star_age
[3]=initial_Yinit
[4]=initial_feh
[5]=initial_MLT
[6]=initial_fov
[7]=effective_T
[8]=luminosity
[9]=delta_nu

'''


'\nsmall grid:\n[0]=row\n[1]=model_number\n[2]=star_mass\n[3]=star_age\n[4]=star_feh\n[5]=star_MLT\n[6]=star_fov\n[7]=effective_T\n[8]=luminosity\n[9]=radius\n[10]=log_g\n[11]=surface_h1\n[12]=surface_he4\n[13]=center_h1\n[14]=center_he4\n[15]=log_center_T\n[16]=log_center_Rho\n[17]=he_core_mass\n[18]=delta_nu\n[19]=delta_Pg\n[20]=nu_max\n[21]=nu_0_10\n[22]=nu_0_11\n[23]=nu_0_12\n[24]=nu_0_13\n[25]=nu_0_14\n[26]=nu_0_15\n\nmid grid:\n[0]=row\n[1]=star_mass\n[2]=star_age\n[3]=initial_Yinit\n[4]=initial_feh\n[5]=initial_MLT\n[6]=initial_fov\n[7]=effective_T\n[8]=luminosity\n[9]=delta_nu\n\n'

In [10]:
'''
required columns:
1. track row number: tends to be column 0
2. input data columns
3. output data columns

required inputs:
variable limit ranges
input_data=np.vstack((data[[track_indeces],2],data[[track_indeces],3]))
output_data=np.vstack(([data[[track_indeces],7],data[[track_indeces],8]]))
new=True
epochs=100000
validation_fraction=0.1
nodes=100
Hidden_layers=6
regularization=False
learning_rate=0.001
beta_1=0.9995
beta_2=0.999
epsilon=1e-9
'''     
file = 'grid_0_0'
pref = []
data = []
#file = 'grid_mid_0_0'
def load_file_and_preferences(file):
    '''
    loads the given grid
    returns data, grid preferences (if available) and neural net preferences
    '''
    try:
        data = np.genfromtxt(file+'.csv',delimiter=",",skip_header=1)
    except FileNotFoundError as e:
        data = None
        
    try:
        with open('preferences.txt') as f:
            pref = yaml.load(f, Loader=yaml.FullLoader)
        try:
            preferences = {'grid_pref':pref['Grid'][file],'NN_pref':pref['NeuralNet']}
        except KeyError:
            print("No grid preferences found: please see \'preferences.txt\'")
            preferences = None

    except FileNotFoundError as e:
        preferences = None
        
    return data,preferences

def create_data_dict(data,pref):
    input_cols = set().union(*(d.keys() for d in [pref['grid_pref']['inputs']]))
    output_cols = set().union(*(d.keys() for d in [pref['grid_pref']['outputs']]))
    data_dict = {'inputs':{},'outputs':{}}
    for i in input_cols:
            data_dict['inputs'][i] = data[:,pref['grid_pref']['inputs'][i]['col']]
    for i in output_cols:
            data_dict['outputs'][i] = data[:,pref['grid_pref']['outputs'][i]['col']]
    return data_dict,pref

def find_tracks(data): #splits data into the individual stellar evalution tracks
    track_starts = np.where(data[:,0] == 0)[0] #finds every occurance of the first row of a evolutionary track
    track_ends = np.append(track_starts[1:]-1,len(data[:,0])-1) #looks for the row elemnt before each first row of each evolutionary track (except for the 0th row) to find the end of the previous track and then the last row of the data is appended to get the end of the last track.
    tracks = list(zip(track_starts,track_ends)) #combines the start and end row element of each track
    return tracks #each element of the list is track[i][0]=start row of ith track, track[i][1]= end row ith track.

def limit_parameter_range(data,pref): #allows the ability to optimize computation by training only on a limited range on multiple variables to just that is useful for the current scientific enquiry.
    tracks = find_tracks(data=data)
    limited_tracks = []
    
    for i in range(len(tracks)):
        track = data[:,3][tracks[i][0]:tracks[i][1]+1]
        lb = (np.abs(track - lower_bounds)).argmin() + tracks[i][0]
        ub = (np.abs(track - upper_bounds)).argmin()+ tracks[i][0]
        if data[:,3][ub] > lower_bounds: #makes stars that are really young aren't included if their maximum age is less than the lower bounds
            limited_tracks.append([lb,ub])
    limited_trackes_indeces = []
    for i in range(len(limited_tracks)):
        limited_trackes_indeces+=list(range(limited_tracks[i][0],limited_tracks[i][1]+1))
        
    return limited_tracks,limited_trackes_indeces

data,pref = load_file_and_preferences(file=file)
if data.any() != None and pref != None:
    data,pref = create_data_dict(data=data,pref=pref)
else:
    print("data or preferences weren\'t loaded properly")